In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!unzip /content/drive/My\ Drive/spotify.zip

Archive:  /content/drive/My Drive/spotify.zip
replace md5sums? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
import json as js
import pandas as pd
import numpy as np
import os

In [40]:
import zipfile

zip_file_path = '/content/drive/MyDrive/spotify.zip'

In [188]:
from google.colab import files
uploaded = files.upload()

Saving mpd.slice.0-999.json to mpd.slice.0-999.json


In [191]:
filename = "mpd.slice.0-999.json"

with open(filename, "r") as file:
  data = js.load(file)

df_playlists = pd.json_normalize(data,record_path=['playlists'])
df_playlists = df_playlists.drop(['collaborative', 'pid', 'modified_at', 'num_followers', 'num_edits', 'description'], axis=1)

df_tracks = pd.json_normalize(data=data['playlists'], record_path='tracks', 
                            meta=['artist_name','track_uri', 'artist_uri', 'duration_ms', 'name'], errors='ignore', record_prefix='track_' )
df_tracks  = df_tracks .drop(['artist_name', 'track_uri', 'artist_uri', 'track_album_uri', 'track_album_name', 'duration_ms'], axis=1)

In [274]:
artists = [
    'Zaz',
    'Oumou',
    'Kendrick Lamar',
    'Pink',
    'Muse',
    'The Killers',
    'Eminem',
    'Taylor Swift',
    'Harry Styles',
    'Lara Fabian',
    'Andrea Bocelli',
    'Laurent Garnier'
]

# Extracting all the artist ID's

artists_to_be_removed = ['Zaz', 'Oumou', 'Pink', 'Lara Fabian', 'Laurent Garnier']
artists = [art for art in artists if art not in artists_to_be_removed]

def get_artist_ID(artists, df_tracks):
  artist_ids = []
  for artist in artists:
    dummy = str(set(df_tracks[df_tracks['track_artist_name'] == artist]['track_artist_uri'].tolist()))
    dummy = dummy[2:len(dummy) - 2]
    artist_ids.append(dummy)

  artists_to_be_removed = ['Zaz', 'Oumou', 'Pink', 'Lara Fabian', 'Laurent Garnier']
  return artist_ids


artists_and_IDs = {artists[i]: get_artist_ID(artists,df_tracks)[i] for i in range(len(artists))}                                   # creating a dictionary out of our results for better visualization
artists_and_IDs = dict(sorted(artists_and_IDs.items(), key = lambda x: x[1], reverse = True))                               # ordering it based on descending order to find the best artists
artists_and_IDs

{'Eminem': 'spotify:artist:7dGJo4pcD2V6oG8kP0tJRR',
 'Harry Styles': 'spotify:artist:6KImCVD70vtIoJWnq6nGn3',
 'Andrea Bocelli': 'spotify:artist:3EA9hVIzKfFiQI0Kikz2wo',
 'Kendrick Lamar': 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg',
 'Muse': 'spotify:artist:12Chz98pHFMPJEknJQMWvI',
 'The Killers': 'spotify:artist:0C0XlULifJtAgn6ZNCW2eu',
 'Taylor Swift': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}

In [275]:
# Find the top artist in the playlist! 
# The artist can stand out by number of appearances_playlists, tracks_featured, or duration
def find_top_artists(artists_and_IDs, argument, df_tracks):
  dummy_list = []                         # a helper list to store data
  for key, value in artists_and_IDs.items():
    if argument == 'appearances_playlists':
      dummy = len(list(set(df_tracks[df_tracks['track_artist_uri'] == value]['name'].tolist())))      # dummy is a placeholder for the data extracted from the dataframe
    elif argument == 'tracks_featured':
      dummy = len(df_tracks[df_tracks['track_artist_uri'] == value]['track_artist_uri'].tolist())
    elif argument == 'duration':
      dummy = sum(df_tracks[df_tracks['track_artist_uri'] == value]['track_duration_ms'].tolist())/60000
    dummy_list.append(dummy)
  return dummy_list


In [285]:
top_artists = pd.DataFrame(artists, columns=['Artists'])
top_artists['Playlist Features'] = 0
top_artists['Track Features'] = 0
top_artists['Duration'] = 0
top_artists

,Artists,Playlist Features,Track Features,Duration
0,Kendrick Lamar,0,0,0
1,Muse,0,0,0
2,The Killers,0,0,0
3,Eminem,0,0,0
4,Taylor Swift,0,0,0
5,Harry Styles,0,0,0
6,Andrea Bocelli,0,0,0


In [286]:
zip_file = zipfile.ZipFile(zip_file_path)
zip_files = {text_file for text_file in zip_file.namelist() if text_file.endswith('.json')}

def extract_data(zip_files, artists, chunk_size, files_done_so_far):
  tops = []

  for f in zip_files:
     if f not in files_done_so_far: 
           with zip_file.open(f) as file:
             js_file = file.read()
             data = js.loads(js_file)

             files_done_so_far.append(f)

             df_tracks = pd.json_normalize(data=data['playlists'], record_path='tracks', 
                                            meta=['artist_name','track_uri', 'artist_uri', 'duration_ms', 'name'], errors='ignore', record_prefix='track_' ).drop(['artist_name', 'track_uri', 'artist_uri', 'track_album_uri', 'track_album_name', 'duration_ms'], axis=1)

               
             top_artists['Playlist Features'] +=  find_top_artists(artists_and_IDs,'appearances_playlists', df_tracks)
             top_artists['Track Features'] += find_top_artists(artists_and_IDs, 'tracks_featured', df_tracks)
             top_artists['Duration'] += find_top_artists(artists_and_IDs, 'duration', df_tracks)
             
             chunk_size += -1
             if chunk_size > 0:
              print("file {} done, chunk {}".format(f, chunk_size))
             else:
              return files_done_so_far

In [287]:
files_done_so_far = extract_data(zip_files, artists, 30, files_done)

file data/mpd.slice.852000-852999.json done, chunk 29
file data/mpd.slice.389000-389999.json done, chunk 28
file data/mpd.slice.457000-457999.json done, chunk 27
file data/mpd.slice.12000-12999.json done, chunk 26
file data/mpd.slice.933000-933999.json done, chunk 25
file data/mpd.slice.115000-115999.json done, chunk 24
file data/mpd.slice.52000-52999.json done, chunk 23
file data/mpd.slice.240000-240999.json done, chunk 22
file data/mpd.slice.816000-816999.json done, chunk 21
file data/mpd.slice.730000-730999.json done, chunk 20
file data/mpd.slice.93000-93999.json done, chunk 19
file data/mpd.slice.414000-414999.json done, chunk 18
file data/mpd.slice.792000-792999.json done, chunk 17
file data/mpd.slice.501000-501999.json done, chunk 16
file data/mpd.slice.218000-218999.json done, chunk 15
file data/mpd.slice.639000-639999.json done, chunk 14
file data/mpd.slice.627000-627999.json done, chunk 13
file data/mpd.slice.198000-198999.json done, chunk 12
file data/mpd.slice.517000-517999.

In [288]:
top_artists

,Artists,Playlist Features,Track Features,Duration
0,Kendrick Lamar,2135,9013,42761.729850
1,Muse,568,1018,4583.602400
2,The Killers,66,122,492.562400
3,Eminem,3385,10556,44184.164533
4,Taylor Swift,551,989,4445.749883
5,Harry Styles,1338,2367,9321.604117
6,Andrea Bocelli,1229,3043,11801.356600


In [289]:
sz = len(zip_file.namelist())
sz

1012

In [290]:
files_done_so_far

['data/mpd.slice.281000-281999.json',
 'data/mpd.slice.661000-661999.json',
 'data/mpd.slice.905000-905999.json',
 'data/mpd.slice.995000-995999.json',
 'data/mpd.slice.312000-312999.json',
 'data/mpd.slice.70000-70999.json',
 'data/mpd.slice.316000-316999.json',
 'data/mpd.slice.191000-191999.json',
 'data/mpd.slice.502000-502999.json',
 'data/mpd.slice.238000-238999.json',
 'data/mpd.slice.92000-92999.json',
 'data/mpd.slice.1000-1999.json',
 'data/mpd.slice.476000-476999.json',
 'data/mpd.slice.253000-253999.json',
 'data/mpd.slice.87000-87999.json',
 'data/mpd.slice.143000-143999.json',
 'data/mpd.slice.74000-74999.json',
 'data/mpd.slice.469000-469999.json',
 'data/mpd.slice.394000-394999.json',
 'data/mpd.slice.38000-38999.json',
 'data/mpd.slice.374000-374999.json',
 'data/mpd.slice.852000-852999.json',
 'data/mpd.slice.389000-389999.json',
 'data/mpd.slice.457000-457999.json',
 'data/mpd.slice.12000-12999.json',
 'data/mpd.slice.933000-933999.json',
 'data/mpd.slice.115000-1159

In [293]:
while len(files_done_so_far) != sz:
  next_files = extract_data(zip_files, artists, 10, files_done_so_far)
  files_done_so_far.append(next_files)

file data/mpd.slice.834000-834999.json done, chunk 9
file data/mpd.slice.439000-439999.json done, chunk 8
file data/mpd.slice.931000-931999.json done, chunk 7
file data/mpd.slice.586000-586999.json done, chunk 6


KeyboardInterrupt: ignored